In [1]:
%run helper_functions.ipynb

No duplicates found.
{'Stockholm': (59.3294, 18.0686), 'Gothenburg': (57.7075, 11.9675), 'Malmö': (55.6058, 13.0358), 'Uppsala': (59.8601, 17.64), 'Norrköping': (58.6, 16.2), 'Västerås': (59.6161, 16.5528), 'Örebro': (59.2739, 15.2075), 'Linköping': (58.4158, 15.6253), 'Helsingborg': (56.05, 12.7167), 'Jönköping': (57.7828, 14.1606), 'Vimmerby': (57.6667, 15.85), 'Sundsvall': (62.4, 17.3167), 'Gävle': (60.6747, 17.1417), 'Umeå': (63.825, 20.2639), 'Karlstad': (59.3783, 13.5042), 'Södertälje': (59.1958, 17.6281), 'Halmstad': (56.6739, 12.8572), 'Eskilstuna': (59.3708, 16.5097), 'Karlskrona': (56.1608, 15.5861), 'Växjö': (56.8769, 14.8092), 'Borås': (57.7211, 12.9403), 'Täby': (59.4333, 18.0833), 'Trollhättan': (58.2828, 12.2892), 'Östersund': (63.1792, 14.6358), 'Luleå': (65.5844, 22.1539), 'Upplands Väsby': (59.5167, 17.9167), 'Trelleborg': (55.3667, 13.1667), 'Kalmar': (56.6614, 16.3628), 'Lidköping': (58.5, 13.1833), 'Skövde': (58.3833, 13.85), 'Nyköping': (58.7531, 17.0086), 'Tumba'

### Function to get initial solution 

In [2]:
def nearest_neighbor(cities, distances):
    """
    Finds a tour (initial solution) by repeatedly visiting the nearest unvisited city.

    Args:
        cities (list): A list of city names to visit.
        distances (dict): A nested dictionary of distances between each pair of cities.

    Returns:
        list: An ordered list representing the tour of cities.
    """
    print("started nearest_neighbor")
    start_city = random.choice(cities)
    unvisited = set(cities)
    unvisited.remove(start_city)
    tour = [start_city]
    while unvisited:
        last_city = tour[-1]
        next_city = min(unvisited, key=lambda city: distances[last_city][city])
        unvisited.remove(next_city)
        tour.append(next_city)
    print("finished nearest_neighbor")
    return tour

### Functions to Improve initial solution

In [3]:
def tabu_search(
    initial_solution,
    distances,
    tenure,
    neighborhood_size,
    initial_cost,
):
    """
    Performs the tabu search algorithm to find an improved solution to the TSP.

    Args:
        initial_solution (list): An ordered list of city names representing the initial tour.
        distances (dict): A nested dictionary of distances between each pair of cities.
        tenure (int): The number of iterations for which moves are tabu.
        neighborhood_size (int): The size of the neighborhood to consider for moves.
        initial_cost (float): The cost of the initial solution in kilometers.

    Returns:
        tuple: A tuple containing the best solution found and its cost.
    """
    best_solution = initial_solution[:]
    best_cost = initial_cost
    tabu_list = []
    current_solution = initial_solution

    for _ in range(100):  # Fixed number of iterations
        neighbors = get_neighborhood(
            current_solution, neighborhood_size
        )  # Adjusted to use neighbors
        best_neighbor = None
        best_neighbor_cost = float("inf")

        for neighbor in neighbors:
            if (
                tuple(neighbor)
                in tabu_list  # Use tuple(neighbor) to make it hashable for tabu list check
            ):
                continue

            current_cost = calculate_cost(neighbor, distances)
            if current_cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = current_cost

        # Move to the best neighbor if it's better than the current best solution
        if best_neighbor and best_neighbor_cost < best_cost:
            current_solution = best_neighbor
            best_solution = best_neighbor[:]
            best_cost = best_neighbor_cost
            # Update the tabu list with the move
            tabu_list.append(
                tuple(
                    current_solution
                )  # Use tuple(neighbor) to make it hashable for tabu list check
            )
            if len(tabu_list) > tenure:
                tabu_list.pop(0)

    return best_solution, best_cost


def get_neighborhood(solution, size):
    """
    Generates a neighborhood of solutions by swapping two cities in the given solution.

    Args:
        solution (list): An ordered list of city names representing the tour.
        size (int): The desired size of the neighborhood.

    Returns:
        list: A list of neighbor solutions.
    """

    neighbors = []
    attempts = 0
    while len(neighbors) < size and attempts < size:
        i, j = sorted(random.sample(range(len(solution)), 2))
        neighbor = solution[:]
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        if neighbor not in neighbors:
            neighbors.append(neighbor)
        attempts += 1
    return neighbors

In [4]:
def solve_tsp(
    selected_cities,
    instance_size,
    tenure,
    neighborhood_size,
    initial_solution,
    initial_cost,
    execution_time_nn,
    distances,
    selected_coordinates,
    dbscan_results,
    execution_time_DBSCAN,
):
    """
    Solves the Traveling Salesman Problem for a set of cities using tabu search after getting the initial solution.

    Args:
        selected_cities (list): A list of city names to be considered.
        instance_size (int): The number of cities in the instance.
        tenure (int): The tabu tenure for the tabu search algorithm.
        neighborhood_size (int): The size of the neighborhood
        the rest of the args are only there to be included in the final data returned

    Returns:
        dict: A dictionary containing various metrics and results from solving the TSP.
    """

    start_time_tabu = time.time()
    improved_solution, improved_cost = tabu_search(
        initial_solution,
        distances,
        tenure,
        neighborhood_size,
        initial_cost,
    )
    print("improved solution: " + str(improved_solution))
    execution_time_tabu = time.time() - start_time_tabu



    return {
        "instance_size": instance_size,
        "tenure": tenure,
        "hood_size": neighborhood_size,
        "selecetd_cities": selected_cities,
        "selected_coordinates": selected_coordinates,
        "initial_sol": initial_solution,
        "improved_sol": improved_solution,
        "initial_cost_km": initial_cost,
        "improved_cost_km": improved_cost,
        "improvement": initial_cost - improved_cost,
        **dbscan_results,
        "dbscan_exec_time": execution_time_DBSCAN,
        "nn_exec_time": execution_time_nn,
        "tabu_exec_time": execution_time_tabu,
    }

### Create Dataset from created instances

In [5]:
def run_experiments(num_experiments=10):
    """
    Runs a series of experiments to solve the Traveling Salesman Problem (TSP) using a combination of algorithms.
    It dynamically adjusts parameters based on the instance size, performs clustering, and then attempts to find
    the optimal tour among selected cities. The results are saved to a CSV file.

    Args:
        num_experiments (int): The number of experiments to run.

    Description:
        - For each experiment, a random set of cities is selected based on the instance size.
        - Tenure and neighborhood size are dynamically set based on the instance size.
        - Distances between cities are fetched and cached to minimize API calls.
        - An initial solution using the nearest neighbor method is calculated.
        - DBSCAN clustering is performed to provide additional insights.
        - A tabu search algorithm is applied to try to improve the initial solution.
        - Results are collected, including execution times and the costs of solutions.
        - Results are appended to a CSV file if it exists; otherwise, a new file is created.
    """
    instance_sizes = range(10, 117)
    all_results = []
    for _ in range(num_experiments):
        instance_size = random.choice(list(instance_sizes))
        selected_cities = random.sample(list(cities_coordinates.keys()), instance_size)
        print("selected cities: " + str(selected_cities))

        # Dynamic ranges based on the instance size
        tenures = range(5, max(16, int(np.sqrt(instance_size) * 3)))
        neighborhood_sizes = range(5, max(16, int(instance_size / 2)))

        # Initialize variables to store the best results for this set of cities
        best_cost = float("inf")
        best_result = None

        # Get all possible distances between cities
        distances = fetch_distances(selected_cities)

        start_time_nn = time.time()
        # Get an innitial solution
        initial_solution = nearest_neighbor(selected_cities, distances)
        print("initial solution: " + str(initial_solution))
        initial_cost = calculate_cost(initial_solution, distances)
        execution_time_nn = time.time() - start_time_nn

        # Getting the coordinates for the cities in the map
        selected_coordinates = {city: cities_coordinates[city] for city in selected_cities}
        print("selected coor: " + str(selected_coordinates))

        start_time_DBSCAN = time.time()
        # DBSCAN clustering and metrics calculation
        dbscan_results = dbscan_and_metrics(selected_coordinates)
        execution_time_DBSCAN = time.time() - start_time_DBSCAN

        # Try different tenures and neighborhood sizes
        for tenure in tenures:
            for neighborhood_size in neighborhood_sizes:
                current_result = solve_tsp(
                    selected_cities,
                    instance_size,
                    tenure,
                    neighborhood_size,
                    initial_solution,
                    initial_cost,
                    execution_time_nn,
                    distances,
                    selected_coordinates,
                    dbscan_results,
                    execution_time_DBSCAN,
                )
                if current_result["improved_cost_km"] < best_cost:
                    best_cost = current_result["improved_cost_km"]
                    best_result = current_result

        # After finding the best result for this set of cities, append it to all_results
        if best_result:  # Ensure best_result is not None
            all_results.append(best_result)
        else:
            print("No results for this iteration.")

    df = pd.DataFrame(all_results)

    # Check if the results file already exists
    file_path = "./datasets/TSP_instances.csv"
    if os.path.exists(file_path):
        # File exists, append data without header
        df.to_csv(file_path, mode="a", header=False, index=False)
    else:
        # File does not exist, write with header
        df.to_csv(file_path, index=False)

    print("Experiments completed and saved to tsp_experiments_results.csv")
    print(df.head())

# For running many instances while giving a rest to the computer
# for x in range(0, 500):
#     run_experiments(1)
#     time.sleep(60)
run_experiments(1)

selected cities: ['Lysekil', 'Frösakull', 'Nynäshamn', 'Ljunghusen', 'Norrtälje', 'Luleå', 'Halmstad', 'Kållered', 'Eskilstuna', 'Skanör med Falsterbo', 'Åstorp', 'Södertälje', 'Söderhamn', 'Karlstad', 'Öjersjö', 'Jakobsberg', 'Gävle', 'Östersund', 'Trollhättan', 'Vega', 'Staffanstorp', 'Uppsala', 'Nyköping', 'Jönköping', 'Enköping', 'Saltsjöbaden', 'Sundbyberg', 'Varberg', 'Oskarström', 'Ekeby', 'Tyresö', 'Djursholm', 'Huddinge', 'Älta', 'Karlskrona', 'Malmö', 'Fisksätra', 'Helsingborg', 'Höganäs', 'Oskarshamn', 'Borås', 'Visby', 'Sundsbruk', 'Bålsta', 'Falkenberg', 'Ekerö', 'Örebro', 'Ängelholm', 'Stenungsund', 'Kalmar', 'Lidköping', 'Växjö', 'Västervik', 'Partille', 'Linköping', 'Karlshamn', 'Kungsbacka', 'Karlsborg', 'Bjärred', 'Uddevalla', 'Hyllinge', 'Särö', 'Höllviken', 'Boo', 'Svinningeudd', 'Hässleholm', 'Nolvik', 'Hönö', 'Ystad', 'Vendelsö', 'Västerås', 'Sölvesborg', 'Trelleborg', 'Åkarp', 'Jordbro', 'Sjöberg', 'Vaxholm', 'Mölnlycke', 'Norrköping', 'Gothenburg', 'Falun', 'Tul